HEADQUARTERS

In [2]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

In [3]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

In [4]:
headquarter = pd.read_sql_table('sede', engineSource)
headquarter.drop(columns=["nombre_contacto", "ciudad_id", "cliente_id"], inplace=True)
headquarter.rename(columns={
  "sede_id": "headquarter_id", 
  "nombre": "name", 
  "direccion": "address",
  "telefono": "phone"
  }, inplace=True)
headquarter["saved"] = pd.to_datetime('now')
headquarter

,headquarter_id,name,address,phone,saved
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
5,24,PASTO VITRINA /,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
6,25,PASTO BODEGA 29/,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
7,26,PALMIRA BODEGA 20 /,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
8,32,CLINICA FABILUX,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687
9,33,FORDES PASOANCHOS,Los angeles distrito Latino,310-70000,2024-10-21 16:22:37.840687


In [5]:
headquarter.to_sql("dim_headquarter", engineDestination, index=False, if_exists="replace")

52